In [1]:
## TODO: 
## 1. Get mean and std of dataset - done
## 2. Write a script to plot loss + accuracy graph
## 3. Get FLOPs - done
## 4. Get num layers - done
## ----------------------------------------------------
## 1. Implement differentiable F1 loss function
## 2. Add class weights
## 3. Implement transfer learning part - done
## 4. Implement T-SNE
## 5. Implement gradcam
## 6. Ablation study
## 

In [1]:
import sys
sys.path.append('../../src/')

In [2]:
%load_ext autoreload
%autoreload 2

from multilabel.train import train_model
from model import initialize_model
from utils import set_requires_grad, save_model
from multilabel.data import load_data
from plotting import plot_data_loader
from multilabel.eval import eval_model

In [3]:
import os
import copy
import time
import random
import pickle

import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm import tqdm

from sklearn.metrics import f1_score, confusion_matrix
from numpy.random import shuffle
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import torch.autograd.profiler as tprofiler
import torch.utils.data as td

plt.rcParams["savefig.bbox"] = 'tight'

In [4]:
seed = 42

random.seed(seed)

# pytorch RNGs
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
np.random.seed(seed)

In [5]:
data_dir = "../../data"
images_dir = "../../data/xray8"

In [6]:
# Get best num_workers
# for i in range(97):
#     start = time.time()
#     data_loader = load_data(images_dir,
#                                                                    batch_size = 96, 
#                                                                    input_size = 299, 
#                                                                    norm_arr = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#                                                                    num_workers = i)
#     iter(data_loader['train']).next()[0].shape
#     print(f"{i}: {time.time()-start}")

In [8]:
# Check if GPU is available.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Models options: resnet50, resnet34, inceptionv3, vgg16, mobile_net_v3_large, efficient_net_b1, efficient_net_b0.
model_name = "resnet34"

# Number of classes.
num_classes = 7

# Batch Size.
batch_size = 32

# Epochs to train for.
num_epochs = 100

# Number of workers for data loader.
num_workers = 12

# Imagenet norm array passed as default value.
# norm_arr=([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# Chest x-ray8 training dataset metrics 
norm_arr=([0.4955, 0.4955, 0.4955], [0.2892, 0.2892, 0.2892])


# Feature extract flag: False - Tune the whole model,
#                       True - Update only the reshaped layer parameters.
feature_extract = False

# Use pretrained flag: None - Use random weights
#                      String - Use pretrained weights given by String
use_pretrained = None

# Initialize the model for this run.
model_pyt, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=use_pretrained)

# lr start and end points for training.
lr_start = 0.01
lr_end = 0.001

# Print the model we just instantiated
#print(model_ft)

# Positive class weights.
pos_weight=torch.as_tensor([2., 3., 6., 6., 3., 7., 9.], dtype=torch.float)
pos_weight = pos_weight.to(device)

cuda:0


In [9]:
data_loaders = load_data(images_dir,
                         batch_size = batch_size, 
                         input_size = input_size, 
                         norm_arr = norm_arr,
                         num_workers = num_workers)

In [10]:
data_loaders['train'].dataset.classes

['Effusion',
 'No Finding',
 'Mass',
 'Nodule',
 'Atelectasis',
 'Pneumothorax',
 'Consolidation']

plot_data_loader(data_loader['train'], (2,2))

plot_data_loader(data_loader['test'], (2,2))

plot_data_loader(data_loader['val'], (2,2))

In [11]:
# Send model to GPU
model_pyt = model_pyt.to(device)

# Find parameters to be updated in this run.
# parameters with requires_grad = True.
params_to_update = model_pyt.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_pyt.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_pyt.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer1.2.conv1.weight
	 layer1.2.bn1.weight
	 layer1.2.bn1.bias
	 layer1.2.conv2.weight
	 layer1.2.bn2.weight
	 layer1.2.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer2.2.conv1.weight
	 layer2.2.bn1.weight
	 layer2.2.bn1.bias
	 layer2.2.conv2.weight
	 layer2.2.bn2.weight
	 layer2.2.bn2.bias
	 layer2.3.conv1.weight
	 layer2.3.bn1.weight
	 layer2.3.bn1.

In [12]:
# 17 min 1 epoch - 128 batch size - inception
# Efficientnet b0 - batch 96 - epoch 50 - num_workers 2 - flip, auto cont, sharp - 

In [13]:
# Observe that all parameters are being optimized
optimizer = optim.Adam(params_to_update, lr=lr_start)

# Learning rate scheduler.
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=lr_end,
                                                 last_epoch=-1)


# Setup the loss fxn
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# Train and evaluate
model_pyt, prof, val_history, train_history = train_model(device, model_pyt, data_loaders, 
                                                            optimizer, scheduler,
                                                            criterion, 
                                                            num_epochs=num_epochs,
                                                            num_classes=num_classes,
                                                            is_inception=(model_name=="inceptionv3"),
                                                            profiler=False)

Epoch 1/100
----------
Epoch [1/100], Step [100/640], Loss: 1.0851, Accuracy: 26.14%
Epoch [1/100], Step [200/640], Loss: 1.0239, Accuracy: 26.43%
Epoch [1/100], Step [300/640], Loss: 1.0770, Accuracy: 26.14%
Epoch [1/100], Step [400/640], Loss: 0.9757, Accuracy: 26.86%
Epoch [1/100], Step [500/640], Loss: 1.0909, Accuracy: 25.86%
Epoch [1/100], Step [600/640], Loss: 1.2545, Accuracy: 25.57%
train Loss: 1.0775 Acc: 0.7902
Epoch [1/100], Step [100/640], Loss: 1.1592, Accuracy: 25.29%
val Loss: 1.0421 Acc: 0.8116

Epoch 2/100
----------
Epoch [2/100], Step [100/640], Loss: 1.0436, Accuracy: 25.71%
Epoch [2/100], Step [200/640], Loss: 0.9508, Accuracy: 27.00%
Epoch [2/100], Step [300/640], Loss: 1.0522, Accuracy: 25.57%
Epoch [2/100], Step [400/640], Loss: 1.0842, Accuracy: 25.00%
Epoch [2/100], Step [500/640], Loss: 1.0331, Accuracy: 25.00%
Epoch [2/100], Step [600/640], Loss: 1.0021, Accuracy: 26.00%
train Loss: 1.0416 Acc: 0.7967
Epoch [2/100], Step [100/640], Loss: 1.2231, Accuracy: 2

In [14]:
dataset_name = 'chest_xray_8'

In [15]:
save_model(model_pyt.state_dict(), '../../models/', 
           f'{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_model_weights.pth')

True

In [16]:
eval_metrics = eval_model(device=device, model=model_pyt, test_loader=data_loaders['test'], is_inception=(model_name=="inceptionv3"), num_classes=7)

In [17]:
for i, v in eval_metrics.items():
    print(f"{i}: {v}")

acc: 0.789882882286773
f1: 0.4107650204486378
cm: [[[2818  455]
  [ 474  571]]

 [[2722  551]
  [ 398  647]]

 [[3532  227]
  [ 394  165]]

 [[3180  483]
  [ 497  158]]

 [[2517  754]
  [ 483  564]]

 [[3574  299]
  [ 304  141]]

 [[2995  857]
  [ 175  291]]]
outputs: [[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
targets: [[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [18]:
with open(f'../../models/val_history_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(val_history, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'../../models/train_history_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(train_history, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'../../models/eval_metrics_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(eval_metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)